In [8]:
import pandas as pd
import glob, os
import warnings
warnings.filterwarnings('ignore')

# 문서 군집화

# [실습] : Opinion Review 데이터 세트를 이용한 문서 군집화 수행
자동차, 전자제품 사이트에서 가져온 리뷰 문서

## 1 데이터 파일명과, 리뷰 데이터 가져오기

In [12]:
# pd.set_option('display.max_colwidth', 700)

# 데이터 파일 위치
path = r'C:\cakd7\git\class\study\m8_머신러닝 응용\OpinosisDataset1.0\topics'

# 지정 위치에 있는 모든 하위 .data 파일들의 파일명을 리스트로 취합
all_files = glob.glob(os.path.join(path, "*.data"))
filename_list = []
opinion_text = []

for file_ in all_files:
    # 개별 파일을 읽어서 df로 생성
    df = pd.read_table(file_, index_col = None, header = 0, encoding = 'latin1')
    
    # 마지막 파일명을 가공
    filename_ = file_.split('\\')[-1] # 경로 \\ 표시를 기준으로 마지막 파일명만 추출 
    filename = filename_.split('.')[0] # . data 제거
    
    # 파일명 리스트와 파일 내용 리스트에 파일명과 파일 내용 추가
    filename_list.append(filename)
    opinion_text.append(df.to_string())

    print(filename_)
    print(filename)
    
print('데이터 길이')
print(len(filename_list))
print(len(opinion_text))
filename_list[0]
opinion_text[0]

accuracy_garmin_nuvi_255W_gps.txt.data
accuracy_garmin_nuvi_255W_gps
bathroom_bestwestern_hotel_sfo.txt.data
bathroom_bestwestern_hotel_sfo
battery-life_amazon_kindle.txt.data
battery-life_amazon_kindle
battery-life_ipod_nano_8gb.txt.data
battery-life_ipod_nano_8gb
battery-life_netbook_1005ha.txt.data
battery-life_netbook_1005ha
buttons_amazon_kindle.txt.data
buttons_amazon_kindle
comfort_honda_accord_2008.txt.data
comfort_honda_accord_2008
comfort_toyota_camry_2007.txt.data
comfort_toyota_camry_2007
directions_garmin_nuvi_255W_gps.txt.data
directions_garmin_nuvi_255W_gps
display_garmin_nuvi_255W_gps.txt.data
display_garmin_nuvi_255W_gps
eyesight-issues_amazon_kindle.txt.data
eyesight-issues_amazon_kindle
features_windows7.txt.data
features_windows7
fonts_amazon_kindle.txt.data
fonts_amazon_kindle
food_holiday_inn_london.txt.data
food_holiday_inn_london
food_swissotel_chicago.txt.data
food_swissotel_chicago
free_bestwestern_hotel_sfo.txt.data
free_bestwestern_hotel_sfo
gas_mileage_toyo

"                                                                                                                                                                                                                                                               , and is very, very accurate .\n0                                                                                                                                                                           but for the most part, we find that the Garmin software provides accurate directions, whereever we intend to go .\n1                                                                                                              This function is not accurate if you don't leave it in battery mode say, when you stop at the Cracker Barrell for lunch and to play one of those trangle games with the tees .\n2                                                                                                                                         

In [14]:
# 파일명 리스트와 파일 내용 리스트를 df로 생성
document_df = pd.DataFrame({'filename' : filename_list, 'opinion_text' : opinion_text})
document_df.head()

,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,...
1,bathroom_bestwestern_hotel_sfo,...
2,battery-life_amazon_kindle,...
3,battery-life_ipod_nano_8gb,...
4,battery-life_netbook_1005ha,...


In [15]:
document_df['opinion_text'][0]

"                                                                                                                                                                                                                                                               , and is very, very accurate .\n0                                                                                                                                                                           but for the most part, we find that the Garmin software provides accurate directions, whereever we intend to go .\n1                                                                                                              This function is not accurate if you don't leave it in battery mode say, when you stop at the Cracker Barrell for lunch and to play one of those trangle games with the tees .\n2                                                                                                                                         

## 2. 리뷰 문서 피처 벡터화

TfidfVectorizer 는 Lemmatization 같은 어근 변환 지원을 안함

-> tokenizer 인자에 커스텀 어근 변환 함수를 적용해 어근 변환 수행

In [19]:
# LemNormalize 함수 생성

In [22]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

# 제거할 부분 문장부호 딕셔너리로 저장
# ord : 하나의 문자를 인자로 받고 해당 문자에 해당하는 유니코드 정수화
# string.punctuation : 따옴표, 마침표 물음표 등의 문장 부호
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation) # 문장부호에있는 부호의 유니코드 정수화 값을 None값으로 저장
print(remove_punct_dict)

# 표제어 추출: 어간 추출과 달리 단어의 형태가 적절히 보존되는 양상을 보이는 특징
lemmar = WordNetLemmatizer()

# 입력으로 들어온 token단어들에 대해서 lemmatization 어근 변환. 
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

# 입력으로 문장을 받아서 stop words 제거-> lower : 소문자 변환 -> nltk.word_tokenize : 단어 토큰화 -> lemmatization 어근 변환. 
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict))) # 문장부호 제거

In [25]:
# TF-IDF 형태로 피처 벡터화
from sklearn.feature_extraction.text import TfidfVectorizer

# 객체생성 (토큰화 방법 : LemNormalize 함수 사용)
tfidf_vect = TfidfVectorizer(tokenizer = LemNormalize, stop_words = 'english', ngram_range = (1, 2), min_df = 0.05, max_df = 0.85) 

In [26]:
# 값 대입 (문서를 피처 벡터화)
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])
feature_vect

<51x4611 sparse matrix of type '<class 'numpy.float64'>'
	with 30124 stored elements in Compressed Sparse Row format>

In [38]:
tfidf_vect.fit_transform(document_df['opinion_text'][:5])

<5x7372 sparse matrix of type '<class 'numpy.float64'>'
	with 8161 stored elements in Compressed Sparse Row format>

## 3. 군집화 수행

### 1) 군집 개수 5개로 수행

In [28]:
## 3. 군집화 수행

from sklearn.cluster import KMeans

# 5개의 집합으로 군집화 수행. 
km_cluster = KMeans(n_clusters=5, max_iter = 10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

print(cluster_label)
cluster_centers

[0 1 3 3 3 4 2 2 0 0 4 4 4 1 1 1 2 2 2 4 1 1 2 4 1 2 3 4 1 2 1 1 1 0 0 4 4
 2 1 1 1 4 3 0 4 1 1 0 0 3 4]


array([[0.00857792, 0.        , 0.        , ..., 0.01710556, 0.        ,
        0.        ],
       [0.        , 0.00099499, 0.00174637, ..., 0.        , 0.00183397,
        0.00144581],
       [0.        , 0.00102835, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01282853, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00881133, 0.        , 0.        , ..., 0.00331061, 0.        ,
        0.        ]])

In [29]:
# 군집화 값 컬럼 생성
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,0
1,bathroom_bestwestern_hotel_sfo,...,1
2,battery-life_amazon_kindle,...,3
3,battery-life_ipod_nano_8gb,...,3
4,battery-life_netbook_1005ha,...,3


In [32]:
document_df['cluster_label'].value_counts()

1    16
4    12
2     9
0     8
3     6
Name: cluster_label, dtype: int64

### 1-2) 군집화 결과 확인

In [39]:
for label in range(len(document_df['cluster_label'].value_counts())):
    df = document_df[document_df['cluster_label']==label].sort_values(by = 'filename')
    print(f'Cluster # {label}의 데이터')
    print(df.head(10), '\n')

Cluster # 0의 데이터
                           filename  \
0     accuracy_garmin_nuvi_255W_gps   
8   directions_garmin_nuvi_255W_gps   
9      display_garmin_nuvi_255W_gps   
33   satellite_garmin_nuvi_255W_gps   
34      screen_garmin_nuvi_255W_gps   
43       speed_garmin_nuvi_255W_gps   
47   transmission_toyota_camry_2007   
48     updates_garmin_nuvi_255W_gps   

                                         opinion_text  cluster_label  
0                                                 ...              0  
8                                                 ...              0  
9                                                 ...              0  
33                                                ...              0  
34                                                ...              0  
43                                                ...              0  
47                                                ...              0  
48                                                ...          

- 전자기기와 호텔, 자동차 리뷰가 조금씩 섞여있음 (군집 개수 조절 필요)

### 2) 재군집

In [55]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화
km_cluster = KMeans(n_clusters = 3, max_iter = 10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_

# 군집라벨 컬럼 다시 생성
document_df['cluster_label'] = cluster_label
document_df['cluster_label'].value_counts()

0    25
1    16
2    10
Name: cluster_label, dtype: int64

### 2-2) 재군집 결과 확인

In [56]:
for label in range(len(document_df['cluster_label'].value_counts())):
    df = document_df[document_df['cluster_label']==label].sort_values(by = 'filename')
    print(f'Cluster # {label}의 데이터')
    display(df, '\n')

Cluster # 0의 데이터


,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,0
2,battery-life_amazon_kindle,...,0
3,battery-life_ipod_nano_8gb,...,0
4,battery-life_netbook_1005ha,...,0
5,buttons_amazon_kindle,...,0
8,directions_garmin_nuvi_255W_gps,...,0
9,display_garmin_nuvi_255W_gps,...,0
10,eyesight-issues_amazon_kindle,...,0
11,features_windows7,...,0
12,fonts_amazon_kindle,...,0


'\n'

Cluster # 1의 데이터


,filename,opinion_text,cluster_label
1,bathroom_bestwestern_hotel_sfo,...,1
13,food_holiday_inn_london,...,1
14,food_swissotel_chicago,...,1
15,free_bestwestern_hotel_sfo,...,1
20,location_bestwestern_hotel_sfo,...,1
21,location_holiday_inn_london,...,1
24,parking_bestwestern_hotel_sfo,...,1
28,price_holiday_inn_london,...,1
32,room_holiday_inn_london,...,1
30,rooms_bestwestern_hotel_sfo,...,1


'\n'

Cluster # 2의 데이터


,filename,opinion_text,cluster_label
6,comfort_honda_accord_2008,...,2
7,comfort_toyota_camry_2007,...,2
16,gas_mileage_toyota_camry_2007,...,2
17,interior_honda_accord_2008,...,2
18,interior_toyota_camry_2007,...,2
22,mileage_honda_accord_2008,...,2
25,performance_honda_accord_2008,...,2
29,quality_toyota_camry_2007,...,2
37,seats_honda_accord_2008,...,2
47,transmission_toyota_camry_2007,...,2


'\n'

[군집 결과]

군집 0 : 전자기기 리뷰

군집 1 : 호텔 리뷰

군집 2 : 자동차 리뷰

#### 과제 1020_1. 상기 군집별 핵심 단어 10개를 아래와 같은 형태로 출력
- 개별 군집번호, 핵심단어-중요도 순서, 파일명

## 4. 군집별 핵심 단어 추출

clusters_centers_ :  단어 피처가 군집의 중심을 기준으로 얼마나 가깝게 위치해 있는지 제공

[출력 형태]

배열 값

행 : 개별 군집 <br>
열 : 개별 피처 _ (개별 군집 내의 상대 위치를 값으로 표현한 좌표 값_ 0~1까지의 값, 1에 가까울 수록 중심과 가까운 값)

In [57]:
# 위에서 군집화한 KMeans 객체 활용
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers 모양 :', cluster_centers.shape)
print(cluster_centers)

cluster_centers 모양 : (3, 4611)
[[0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]]


-> 3행 4611열 배열 형태 (군집이 3개, 단어 피처가 4611개로 구성됨)

In [60]:
# argsort()[:, ::-1] : 값이 큰 순서로 정렬된 위치 인덱스 값을 반환
print(cluster_centers.argsort()[:, ::-1])
len(cluster_centers.argsort()[:, ::-1])

[[3572  685 2342 ... 1750 3647 4610]
 [3466 2095 3648 ... 2272 2275    0]
 [2252 3612 2706 ... 2791 2789    0]]


3

In [61]:
# cluster_centers 배열 값이 큰 순으로 정렬된 인덱스 값 반환
centroid_feature_ordered_ind = cluster_centers.argsort()[:, ::-1]

centroid_feature_ordered_ind[0, :10] # 해당 군집(0)의 top 10 피처 단어의 인덱스 위치 

array([3572,  685, 2342,  690, 2443, 2357, 1353, 4306, 3736, 4330],
      dtype=int64)

In [69]:
# 위에서 벡터화한 객체에서 피처명 리스트 추출
# get_feature_names() 사용 
feature_names_ex = tfidf_vect.get_feature_names()
feature_names_ex[:10]

['0 5',
 '0 58',
 '0 garmin',
 '0 love',
 '0 mention',
 '0 paged',
 '0 second',
 '1 0',
 '1 16',
 '1 18']

In [71]:
# 피처명 리스트값의 인덱스가 있는 단어 추출
[feature_names_ex[ind] for ind in centroid_feature_ordered_ind[0, :10]]

['headphone',
 '313 took',
 'compact easy',
 '316',
 'contraint',
 'complainer',
 'annoying le',
 'life screen',
 'im new',
 'light 9']

In [72]:
feature_names_ex[3572]

'headphone'

In [73]:
# 해당 피처 단어의 중심 위치 상댓값 추출
km_cluster.cluster_centers_[0, 3572]

0.13738120192560713

In [74]:
# 해당 단어 딕셔너리형태로 핵심단어, 상대 위치 값을 저장
cluster_details = {}

cluster_details[0] = {}
# 0번 군집
cluster_details[0]['cluster'] = 0
cluster_details[0]['top_features'] = 'headphone' # 0번 군집의 핵심 단어
cluster_details[0]['top_features_value'] = 0.13738120192560713 # 해당 피처 단어의 중심 위치 상대 값 저장
cluster_details

{0: {'cluster': 0,
  'top_features': 'headphone',
  'top_features_value': 0.13738120192560713}}

In [78]:
# 0번 군집의 파일이름 
document_df[document_df['cluster_label']==0]['filename'].values.tolist()

['accuracy_garmin_nuvi_255W_gps',
 'battery-life_amazon_kindle',
 'battery-life_ipod_nano_8gb',
 'battery-life_netbook_1005ha',
 'buttons_amazon_kindle',
 'directions_garmin_nuvi_255W_gps',
 'display_garmin_nuvi_255W_gps',
 'eyesight-issues_amazon_kindle',
 'features_windows7',
 'fonts_amazon_kindle',
 'keyboard_netbook_1005ha',
 'navigation_amazon_kindle',
 'performance_netbook_1005ha',
 'price_amazon_kindle',
 'satellite_garmin_nuvi_255W_gps',
 'screen_garmin_nuvi_255W_gps',
 'screen_ipod_nano_8gb',
 'screen_netbook_1005ha',
 'size_asus_netbook_1005ha',
 'sound_ipod_nano_8gb',
 'speed_garmin_nuvi_255W_gps',
 'speed_windows7',
 'updates_garmin_nuvi_255W_gps',
 'video_ipod_nano_8gb',
 'voice_garmin_nuvi_255W_gps']

In [79]:
# 위를 한번에 출력해주는 함수 생성 (입력값 : 군집 모델, 군집 값, 피처 단어, 군집 개수, topN (각 군집 별 추출할 인덱스 개수))

def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    # 빈 딕셔너리 (최종 출력 값 : 1. 개별 군집번호, 2. 핵심 단어 3. 핵심단어의 중심 위치 상댓값, 4. 파일명 속성 값 정보)
    cluster_details = {}
    
    # 개별 군집 내의 상대 위치 값이 가까운 값 순으로 정렬 후 해당 값 인덱스 반환
    centorid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:, ::-1]
    
    # 개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값 딕셔너리에 저장
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num # 1. 개별 군집 번호 (군집 id)
        
        # 해당 군집 id 의 top 10 피처 단어의 인덱스 추출 [10개의 리스트 형태]
        top_feature_indexes = centorid_feature_ordered_ind[cluster_num, :top_n_features]
        # 해당 인덱스로 단어 추출 
        top_features = [feature_names[ind] for ind in top_feature_indexes] 
        
        # top_feature_indexes로 해당 피처 단어의 중심 위치 상대값 추출
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist() 
        
        # 출력할 빈 딕셔너리 객체에 개별 군집별 핵심단어와 중심위치 상댓값 입력
        cluster_details[cluster_num]['top_features'] = top_features # 2. 핵심 단어
        cluster_details[cluster_num]['top_features_value'] = top_feature_values # 3. 핵심단어의 중심 위치 상댓값
        
        # 해당 군집의 파일이름 리스트로 저장
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename'] 
        filenames = filenames.values.tolist()
        
        cluster_details[cluster_num]['filenames'] = filenames # 4. 파일명 속성 값 정보
        
    return cluster_details

In [80]:
# 데이터 대입

feature_names = tfidf_vect.get_feature_names() # TF-IDF 변환된 객체에서 피처명 리스트 값 추출

# 함수 적용

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data = document_df, feature_names=feature_names, clusters_num=3, top_n_features=10)

cluster_details

{0: {'cluster': 0,
  'top_features': ['headphone',
   '313 took',
   'compact easy',
   '316',
   'contraint',
   'complainer',
   'annoying le',
   'life screen',
   'im new',
   'light 9'],
  'top_features_value': [0.13738120192560713,
   0.12029588637881046,
   0.06416898221290011,
   0.06330587553655656,
   0.058609811414901014,
   0.05646823079053855,
   0.05393721234628231,
   0.05389363145594924,
   0.05127615400589759,
   0.05119379623996657],
  'filenames': ['accuracy_garmin_nuvi_255W_gps',
   'battery-life_amazon_kindle',
   'battery-life_ipod_nano_8gb',
   'battery-life_netbook_1005ha',
   'buttons_amazon_kindle',
   'directions_garmin_nuvi_255W_gps',
   'display_garmin_nuvi_255W_gps',
   'eyesight-issues_amazon_kindle',
   'features_windows7',
   'fonts_amazon_kindle',
   'keyboard_netbook_1005ha',
   'navigation_amazon_kindle',
   'performance_netbook_1005ha',
   'price_amazon_kindle',
   'satellite_garmin_nuvi_255W_gps',
   'screen_garmin_nuvi_255W_gps',
   'screen_ipod_n

In [94]:
# 보기 편하게 출력
for cluster_num, cluster_detail in cluster_details.items():
    print(f'----------------------Cluster {cluster_num}----------------------')
    
    print('TOP 10 핵심 단어')
    print(cluster_detail['top_features'], '\n')
    print('TOP 10 핵심 단어의 중심 위치 상댓값')
    print(cluster_detail['top_features_value'], '\n')
    print('Review 파일명')
    print(cluster_detail['filenames'], '\n')

----------------------Cluster 0----------------------
TOP 10 핵심 단어
['headphone', '313 took', 'compact easy', '316', 'contraint', 'complainer', 'annoying le', 'life screen', 'im new', 'light 9'] 

TOP 10 핵심 단어의 중심 위치 상댓값
[0.13738120192560713, 0.12029588637881046, 0.06416898221290011, 0.06330587553655656, 0.058609811414901014, 0.05646823079053855, 0.05393721234628231, 0.05389363145594924, 0.05127615400589759, 0.05119379623996657] 

Review 파일명
['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_kindle', 'directions_garmin_nuvi_255W_gps', 'display_garmin_nuvi_255W_gps', 'eyesight-issues_amazon_kindle', 'features_windows7', 'fonts_amazon_kindle', 'keyboard_netbook_1005ha', 'navigation_amazon_kindle', 'performance_netbook_1005ha', 'price_amazon_kindle', 'satellite_garmin_nuvi_255W_gps', 'screen_garmin_nuvi_255W_gps', 'screen_ipod_nano_8gb', 'screen_netbook_1005ha', 'size_asus_netbook_1005ha', 'sound_ipod

[출력]

군집 0 : headphone, 